In [23]:
import os
import time

In [24]:
#Size of the file
os.path.getsize('C:/Users/HAZARIKA/Desktop/Rate.csv')

1968829442

In [25]:
#Read in the data with Dask

In [26]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('C:/Users/HAZARIKA/Desktop/Rate.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.011967897415161133 sec


In [27]:
#Read in the data with Pandas

In [28]:
import pandas as pd
start = time.time()
df = pd.read_csv('C:/Users/HAZARIKA/Desktop/Rate.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  28.48867106437683 sec


In [29]:
# Read in the data with Modin and Ray

In [30]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('C:/Users/HAZARIKA/Desktop/Rate.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

2021-04-16 13:16:20,499	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


Read csv with modin and ray:  24.314270734786987 sec


In [ ]:
# Here Dask is better than Pandas, Modin and Ray, with the least reading time of 0.012 sec

In [31]:
from dask import dataframe as dd
df = dd.read_csv('C:/Users/HAZARIKA/Desktop/Rate.csv',delimiter=',')

In [32]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 24 entries, BusinessYear to RowNumber
dtypes: object(10), float64(9), int64(5)

In [33]:
#No. of Rows
len(df.index)

12694445

In [34]:
#No, of Columns
len(df.columns)

24

In [35]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [36]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [37]:
data=df.columns
data

Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'VersionNum',
       'ImportDate', 'IssuerId2', 'FederalTIN', 'RateEffectiveDate',
       'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age',
       'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents',
       'RowNumber'],
      dtype='object')

In [54]:
#Validation

In [55]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [56]:
%%writefile utility.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [43]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: Rate
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - BusinessYear
      - StateCode
      - IssuerId
      - SourceName
      - VersionNum
      - ImportDate
      - IssuerId2
      - FederalTIN
      - RateEffectiveDate
      - RateExpirationDate
      - PlanId
      - RatingAreaId
      - Tobacco
      - Age
      - IndividualRate
      - IndividualTobaccoRate
      - Couple
      - PrimarySubscriberAndOneDependent
      - PrimarySubscriberAndTwoDependents
      - PrimarySubscriberAndThreeOrMoreDependents
      - CoupleAndOneDependent
      - CoupleAndTwoDependents
      - CoupleAndThreeOrMoreDependents
      - RowNumber

Overwriting store.yaml


In [46]:
# Reading config file
import utility as util
config_data = util.read_config_file("store.yaml")

In [47]:
#data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'Rate',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['BusinessYear - StateCode - IssuerId - SourceName - VersionNum - ImportDate - IssuerId2 - FederalTIN - RateEffectiveDate - RateExpirationDate - PlanId - RatingAreaId - Tobacco - Age - IndividualRate - IndividualTobaccoRate - Couple - PrimarySubscriberAndOneDependent - PrimarySubscriberAndTwoDependents - PrimarySubscriberAndThreeOrMoreDependents - CoupleAndOneDependent - CoupleAndTwoDependents - CoupleAndThreeOrMoreDependents - RowNumber']}

In [48]:
# Reading process of the file using Dask
from dask import dataframe as dd
df_sample = dd.read_csv('C:/Users/HAZARIKA/Desktop/Rate.csv',delimiter=',')
df_sample.head()

,BusinessYear,StateCode,IssuerId,SourceName,VersionNum,ImportDate,IssuerId2,FederalTIN,RateEffectiveDate,RateExpirationDate,...,IndividualRate,IndividualTobaccoRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,RowNumber
0,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,29.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
1,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,14
2,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,15
3,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
4,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16


In [49]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = "C:/Users/HAZARIKA/Desktop/" + config_data['file_name'] + f'.{file_type}'

In [50]:
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,BusinessYear,StateCode,IssuerId,SourceName,VersionNum,ImportDate,IssuerId2,FederalTIN,RateEffectiveDate,RateExpirationDate,...,IndividualRate,IndividualTobaccoRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,RowNumber
0,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,29.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
1,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,14
2,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,15
3,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
4,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16


In [51]:
#validating the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['planid', 'rateexpirationdate', 'rateeffectivedate', 'importdate', 'primarysubscriberandthreeormoredependents', 'rownumber', 'federaltin', 'issuerid', 'individualrate', 'issuerid2', 'age', 'ratingareaid', 'couple', 'individualtobaccorate', 'statecode', 'coupleandtwodependents', 'tobacco', 'sourcename', 'primarysubscriberandonedependent', 'coupleandthreeormoredependents', 'businessyear', 'coupleandonedependent', 'primarysubscriberandtwodependents', 'versionnum']
Following YAML columns are not in the file uploaded ['businessyear - statecode - issuerid - sourcename - versionnum - importdate - issuerid2 - federaltin - rateeffectivedate - rateexpirationdate - planid - ratingareaid - tobacco - age - individualrate - individualtobaccorate - couple - primarysubscriberandonedependent - primarysubscriberandtwodependents - primarysubscriberandthreeormoredependents - coupleandonedependent - coupleandtw

0

In [52]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['businessyear', 'statecode', 'issuerid', 'sourcename', 'versionnum',
       'importdate', 'issuerid2', 'federaltin', 'rateeffectivedate',
       'rateexpirationdate', 'planid', 'ratingareaid', 'tobacco', 'age',
       'individualrate', 'individualtobaccorate', 'couple',
       'primarysubscriberandonedependent', 'primarysubscriberandtwodependents',
       'primarysubscriberandthreeormoredependents', 'coupleandonedependent',
       'coupleandtwodependents', 'coupleandthreeormoredependents',
       'rownumber'],
      dtype='object')
columns of YAML are: ['BusinessYear - StateCode - IssuerId - SourceName - VersionNum - ImportDate - IssuerId2 - FederalTIN - RateEffectiveDate - RateExpirationDate - PlanId - RatingAreaId - Tobacco - Age - IndividualRate - IndividualTobaccoRate - Couple - PrimarySubscriberAndOneDependent - PrimarySubscriberAndTwoDependents - PrimarySubscriberAndThreeOrMoreDependents - CoupleAndOneDependent - CoupleAndTwoDependents - CoupleAndThre

In [53]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation failed
Following File columns are not in the YAML file ['planid', 'rateexpirationdate', 'rateeffectivedate', 'importdate', 'primarysubscriberandthreeormoredependents', 'rownumber', 'federaltin', 'issuerid', 'individualrate', 'issuerid2', 'age', 'ratingareaid', 'couple', 'individualtobaccorate', 'statecode', 'coupleandtwodependents', 'tobacco', 'sourcename', 'primarysubscriberandonedependent', 'coupleandthreeormoredependents', 'businessyear', 'coupleandonedependent', 'primarysubscriberandtwodependents', 'versionnum']
Following YAML columns are not in the file uploaded ['businessyear - statecode - issuerid - sourcename - versionnum - importdate - issuerid2 - federaltin - rateeffectivedate - rateexpirationdate - planid - ratingareaid - tobacco - age - individualrate - individualtobaccorate - couple - primarysubscriberandonedependent - primarysubscriberandtwodependents - primarysubscriberandthreeormoredependents - coupleandonedependent - coupleandtw

In [39]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('C:/Users/HAZARIKA/Desktop/Rate.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('Rate.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['C:/Users/HAZARIKA/Rate.csv.gz\\00.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\01.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\02.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\03.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\04.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\05.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\06.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\07.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\08.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\09.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\10.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\11.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\12.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\13.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\14.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\15.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\16.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\17.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\18.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\19.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\20.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\21.part',
 'C:/Users/HAZARIKA/Rate.csv.gz\\22.part',
 'C:/Users/

In [40]:
#number of files in gz format folder
import os
entries = os.listdir('C:/Users/HAZARIKA/Rate.csv.gz/')
for entry in entries:
    print(entry)

00.part
01.part
02.part
03.part
04.part
05.part
06.part
07.part
08.part
09.part
10.part
11.part
12.part
13.part
14.part
15.part
16.part
17.part
18.part
19.part
20.part
21.part
22.part
23.part
24.part
25.part
26.part
27.part
28.part
29.part
30.part


In [41]:
#size of the gz format folder
os.path.getsize('C:/Users/HAZARIKA/Rate.csv.gz')

8192

2021-04-16 14:59:07,403	WARNING worker.py:1034 -- The node with node id a00bbce0ebae923a5bbde0f3a08f7fb9f646207b has been marked dead because the detector has missed too many heartbeats from it. This can happen when a raylet crashes unexpectedly or has lagging heartbeats.
(pid=None) [2021-04-16 14:59:11,757 C 15096 14056] node_manager.cc:746:  Check failed: node_id != self_node_id_ Exiting because this node manager has mistakenly been marked dead by the monitor: GCS didn't receive heartbeats within timeout 30000 ms. This is likely since the machine or raylet became overloaded.
(pid=None) [2021-04-16 14:59:12,375 E 15096 14056] logging.cc:414: *** Aborted at 1618570752 (unix time) try "date -d @1618570752" if you are using GNU date ***
(pid=None) [2021-04-16 15:00:13,327 E 15096 14056] logging.cc:414:     @     0x7ffbf4731881 raise
(pid=None) [2021-04-16 15:00:13,632 E 15096 14056] logging.cc:414:     @     0x7ffbf4732851 abort
(pid=None) [2021-04-16 15:00:14,257 E 15096 14056] logging.